In [9]:
# Importing The Necessary Libraries
import pandas as pd
import numpy as np
np.NaN = np.nan # For pandas_ta
import yfinance as yf
import pandas_datareader as pdr
import pandas_ta as ta
import datetime 
from datetime import timedelta

In [2]:
CONTEXT_LEN = 120
SP_TICKER = '^GSPC'
DJ_TICKER = '^DJI'

In [12]:
today = datetime.date.today()
previous_120_days = today - timedelta(days=120)

In [13]:
def get_stock_data(ticker, start_date, end_date):
    copy_df = yf.download(ticker, start=start_date, end=end_date).ffill().bfill()
    copy_df = copy_df.reset_index()
    copy_df = copy_df[['Date','Close']]
    copy_df.columns = copy_df.columns.get_level_values(0)
    copy_df = copy_df.rename(columns={
           'Date'  : 'DATE',
           'Close' : f'{ticker} Close'
    })
    return copy_df

In [16]:
SP_data = get_stock_data(SP_TICKER, previous_120_days, today)
DJ_data = get_stock_data(DJ_TICKER, previous_120_days, today)

[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


In [17]:
def add_data_based_on_date(main_df, new_data, col_name):
    return pd.merge_asof(
            main_df, 
            new_data,
            on='DATE',
            direction='forward' # How we fill the gaps when adding content
        )[col_name]

In [ ]:
combined_df = pd.DataFrame()

combined_df['DATE'] = SP_data['DATE']

# Adding Stock Markets
combined_df[f'{SP_TICKER} Close'] = add_data_based_on_date(combined_df, SP_data, f'{SP_TICKER} Close')
combined_df[f'{DJ_TICKER} Close'] = add_data_based_on_date(combined_df, DJ_data, f'{DJ_TICKER} Close')